# <center>Centro Universitário Facens<br/></center>
<br/>
<font size="4"><center><b>Ciência de Dados</b></center></font>
  
<font size="3"><center>Prof. Dr. Renato Moraes Silva</center></font>
<br/>

# <center>Perceptron multicamadas: implementação do zero</center>

Neste exercício, você irá implementar uma rede neural artificial com *backpropagation* que será aplicada na tarefa de reconhecimento de dígitos manuscritos (Figura 1). 

<center>
<div style="padding: 0px; float: center;">
    <img src="figs/digitos.png"  style="height:400px;"/> 
    <center><em>Figura 1. Amostras do conjunto de dados.</em></center>
</div>
</center>

O conjunto que você utilizará será de digitos manuscritos. Cada imagem tem dimensão de 28 x 28 pixels e cada pixel é representado por um valor inteiro com a intensidade do tom de cinza naquela região.

O conjunto de dados contém 5.000 amostras dos dígitos de 0 a 9.

Instruções
----------

Neste exercício, você precisará completar as seguintes funções:

* `forwardPropagation()`
* `funcaoCusto()`
* `funcaoCusto_reg()`
* `sigmoidGradient()`
* `predicao()`

Você não poderá criar nenhuma outra função. Apenas altere as rotinas fornecidas.

## Carregando e visualizando os dados

Vamos descompactar os dados.

In [ ]:
# -*- coding: utf-8 -*-

import numpy as np  # importa a biblioteca usada para trabalhar com vetores e matrizes
import pandas as pd # importa a biblioteca usada para trabalhar com dataframes (dados em formato de tabela) e análise de dados
import os # importa a biblioteca para tarefas relacionadas ao sistema operacional

# biblioteca para abrir as imagens
!pip install opencv-python
import cv2

from zipfile import ZipFile # biblioteca para arquivoz zipados

z = ZipFile('dados/MNIST.zip', 'r')
z.extractall('dados/')
z.close()

Vamos carregar essa base de dados e guardar dos dados e as classes em uma lista.

In [ ]:
def importDataset(path):
    # percorre as pastas e coleta as imagens 
    labelList = []
    imgList = []
    for i in range(0,10):

        print("Folder: %d" %i)
         
        # coleta os nomes dos arquivos da pasta
        files = os.listdir(path + str(i) )

        for file in files:

            # abre a imagem
            img = cv2.imread(path + str(i) + '/' + file, cv2.IMREAD_GRAYSCALE)
            
            imgList.append(img)
            
            # guarda a classe da imagem
            labelList.append(i)
    
    return imgList, labelList

# Caminho dos arquivos
path = "dados/MNIST/"

imgList, labelList = importDataset(path)       
    
print('\nQuantidade de imagens: ', len(imgList))

print('\nDimensão de cada imagem: ', imgList[0].shape)

As imagens e as classes foram guardadas em uma lista. Iremos converter as imagens em uma matriz, onde cada linha representará uma imagem. Converta também a lista de classes em um vetor. 

In [ ]:
# variável que deverá recer as imagens 
X_img = None

# variável que irá receber as classes
Y = None

for i, img in enumerate(imgList):
    imgList[i] = np.ravel( imgList[i] )
                          

X_img = np.asarray( imgList )
Y = np.asarray( labelList )
        
print('\nDimensao de X: ', X_img.shape)

print('\nDimensao de Y: ', Y.shape)

print('\nClasses do problema: ', np.unique(Y))

Os tons de cinza são representados por valores entre 0 e 255. Esse formato de valores não é adequado para métodos de classificação baseados em otimização. Por isso, você deve normalizar esses valores para o intervalo entre 0 e 1. Para isso, basta dividir por 255.

In [ ]:
# variável que deverá recer as imagens normalizadas
X = None

X = X_img/255.0

print('Maior valor de X: %1.10f' %np.max(X) )
print('Menor valor de X: %1.10f' %np.min(X) )

Vamos plotar aleatoriamente 100 amostras da base de dados.

In [ ]:
import matplotlib.pyplot as plt

def visualizaDados(X):
    
    # Calcula numero de linhas e colunas
    m, n = X.shape
    
    # calcula a largura das imagens
    example_width = int(round(np.sqrt(n)) )
    example_height = int(n / example_width)

    # Calcula numero de itens que serao exibidos
    display_rows = int(np.floor(np.sqrt(m)))
    display_cols = int(np.ceil(m / display_rows))

    fig, axs = plt.subplots(display_rows, display_cols, figsize=(7, 7))
    
    k = 0 # índice da imagem
    for i in range(axs.shape[0]):
        for j in range(axs.shape[1]):
            
            new_X = np.reshape( np.ravel(X[k,:]), (example_width, example_height) )

            axs[i,j].imshow(new_X, cmap='gray'); 
            axs[i,j].axis('off')
            
            k += 1
    
    plt.show()


idx_perm = np.random.permutation( range(X.shape[0]) )
visualizaDados( X[idx_perm[0:100],:] )

Agora, iremos separar os dados em 80% para treinamento e 20% para teste.

In [ ]:
def holdout(X, Y, porcTrain = 0.8, randomSeed = 10):
    
    # total de dados
    m = X.shape[0]
    
    # total de dados para treino
    qtdTreino = int(m * porcTrain)
    
    # embaralha os indices
    indicesEmbaralhados = np.random.RandomState(randomSeed).permutation(m) 
    
    # embaralha os dados
    X_shuffled = X[ indicesEmbaralhados, : ]
    Y_shuffled = Y[ indicesEmbaralhados]
    
    # separa em treino e teste
    X_train = X_shuffled[ 0:qtdTreino,: ]
    Y_train = Y_shuffled[ 0:qtdTreino ]
    
    X_test = X_shuffled[ qtdTreino:,: ]
    Y_test = Y_shuffled[ qtdTreino: ]
    
    return X_train, X_test, Y_train, Y_test

# separa em treino e teste
X_train, X_test, Y_train, Y_test = holdout(X, Y, porcTrain = 0.8, randomSeed = 10)

print('Qtd. dados de treinamento: %d (%1.2f%%)' %(X_train.shape[0], (X_train.shape[0]/X.shape[0])*100) )
print('Qtd. de dados de teste: %d (%1.2f%%)' %(X_test.shape[0], (X_test.shape[0]/X.shape[0])*100) )

## Inicializando os parâmetros

A rede neural proposta para este exercício tem 3 camadas: uma camada de entrada, uma camada oculta e uma camada de saída. É importante lembrar que a camada de entrada possui 784 neurônios que corresponde ao total de pixels de cada imagem (sem considerar o *bias*, sempre +1).

Agora, vamos inicializar os parâmetros mais importantes.

In [ ]:
input_layer_size  = 784  # 28x28 dimensao das imagens de entrada
hidden_layer_size = 25   # 25 neuronios na camada oculta
num_labels = 10          # 10 rotulos, de 1 a 10  
                         #  (observe que a classe "0" recebe o rotulo 10)

print('\nCarregando informações sobre a arquitetura da rede\n')

Como o problema que estamos trabalhando é multiclasse, a camada de saída terá $K$ neurônios, onde $K$ é a quantidade de classes. O vetor de saída precisa ser criado a partir da classe original, tornando-se compatível com a rede neural, ou seja, espera-se vetores com $|C|$ (onde, $|C|$ é o número de classes) posições contendo 1 para o elemento referente à classe esperada e 0 nos demais elementos. Por exemplo, seja 5 o rótulo de determinada amostra, o vetor $Y$ correspondente terá 1 na posição $y_5$ e 0 nas demais posições.

In [ ]:
# converte as classes para o formato binário
def converteBinario(Y, num_labels):
    m = len(Y)
    
    # variavel que irá ser retornada
    Y_bin = None
    
    # (1) Converta em um vetor binário: 1 para o índice que representa a 
    #     classe atual, enquanto atribui 0 aos demais. 

    Y_bin = np.zeros([m,num_labels], dtype=int)
    for i in range(m):
        Y_bin[i,Y[i]] = 1
        
    return Y_bin

# chama a funca que ira converter em binario
Y_trainBin = converteBinario(Y_train, num_labels)

print("Cinco primeiras classes: ")
print(Y_trainBin[0:5, :])

Precisamos criar os pesos iniciais que serão usados na rede neural. Eles podem ser definidos aleatoriamente, com valores entre 0 e 1. Depois, a rede irá melhorando eles durante o aprendizado. Para que toda a execução gere o mesmo resultado, vamos usar uma semente para a função de geração de números aleatórios.

In [ ]:
def inicializaPesosAleatorios(inputSize, outputSize, randomSeed = 10):
    '''
    Inicializa aleatoriamente os pesos de uma camada usando 
    inputSize (conexoes de entrada) e outputSize (conexoes de saida).

    W sera definido como uma matriz de dimensoes [outputSize, 1 + inputSize]
    visto que devera armazenar os termos para "bias".
    
    randomSeed: indica a semente para o gerador aleatorio
    '''
    
    # variável que deve ser retornada
    W = None
    
    # parâmetro usado para controlar a escala dos pesos iniciais
    epsilon_init = 0.12 

    # se for fornecida uma semente para o gerador aleatorio
    if randomSeed is not None:
        W = np.random.RandomState(randomSeed).rand(outputSize, 1 + inputSize) * 2 * epsilon_init - epsilon_init
        
    # se nao for fornecida uma semente para o gerador aleatorio
    else:
         W = np.random.RandomState().rand( outputSize, 1+inputSize ) * 2 * epsilon_init - epsilon_init
             
    return W

initial_Theta1 = inicializaPesosAleatorios(input_layer_size, hidden_layer_size, randomSeed = 10)
initial_Theta2 = inicializaPesosAleatorios(hidden_layer_size, num_labels, randomSeed = 20)

print('\nParametros inicializados com sucesso!!!')

## Calcula o custo (*Feedforward*)

Você precisará implementar a função de custo e gradiente para a rede neural. Mas, antes disso é necessário computar a saída da rede neural ($h_\Theta(x^{(i)})$) usando a etapa *forward propagation*. Nessa etapa, iremos usar a função de ativação sigmoidal: $\displaystyle g(z) = \frac{1}{1 + e^{-z}}$. Portanto, precisamos implementá-la primeiro. 

In [ ]:
def sigmoid(z):
    """
    Calcula a função sigmoidal  
    """
    
    # variável que receberá o valor da função sigmoidal
    sig = None
    
    sig = 1/(1+np.exp(-z))
    
    return sig

# testando a função sigmoidal
z = sigmoid(0)
print('sigmoid de 0 = %1.6f' %(z))

z = sigmoid(10)
print('sigmoid de 10 = %1.6f' %(z))

Agora que implementamos a função de ativação sigmoidal, podemos usá-la para computar a resposta (ativação) de cada neurônio da camada de saída: $h_\Theta(x^{(i)})_k$ = $a^{(3)}_k$, onde $k$ é o índice da $k$-ésima unidade de saída.

Usando multiplicação matricial, a etapa *forward propagation* pode ser resumida da seguinte forma:

$a^{(1)}  = x  \text{ } \left(\text{adicionar } x_0 \right)$ 


$z^{(2)}  =  \Theta^{(1)} a^{(1)}$


$a^{(2)}  = g \left(z^{(2)}\right)  \text{ } \left(\text{adicionar } a_0^{(2)} \right)$ 


$z^{(3)}  =  \Theta^{(2)} a^{(2)} $


$a^{(3)} = h_\Theta(x) = g\left(z^{(3)}\right)$ 


Agora, você deve completar a função que executa a etapa *forward propagation*. O valor retornado por essa etapa ($h_\Theta(x)$), será usado depois para obter o custo. 

In [ ]:
def forwardPropagation(X, Theta1, Theta2):
    '''
    Calcula o custo da rede neural
    voltada para tarefa de classificacao.

    Parametros
    ----------   
    X : matriz com os dados de treinamento
    Theta1: pesos que ligam a camada de entrada à camada intermediária
    Theta2: pesos que ligam a camada intermediária à camada de saída
    
    Retorno
    -------
    hx: saída da rede
    
    '''
    
    # estrai a quantidade de classes
    num_labels = len(np.unique(Y))

    # Qtde de amostras
    m = X.shape[0]
    
    # tamanho da camada intermediaria
    hidden_layer_size = Theta1.shape[0]
     
    # Inicializa a variável que deve ser retornada corretamente
    # Cada dado em X irá gerar uma resposta para cada uma das classes
    hx = np.zeros([m,num_labels])
    
    # variáveis que você terá que retornar
    # --------------------------------------
    # "z" da segunda camada 
    z2 = np.zeros([m,hidden_layer_size])
    
    # "a" da segunda camada
    a2 = np.zeros([m,hidden_layer_size+1])   
    
    # "z" da terceira camada
    z3 = np.zeros([m,num_labels])
    
    # "a" da terceira camada (o mesmo que hx)
    # cada dado em X irá gerar uma resposta para cada uma das classes
    hx = np.zeros([m,num_labels]) 
    # --------------------------------------
    

    ########################## COMPLETE O CÓDIGO AQUI  ########################
    # Instrucoes: Voce deve completar o codigo a partir daqui 
    #
    # (1) Execute a etapa de forward propagation para obter a resposta da rede
    #      para cada dado em X em relação a cada uma das classes
    # (2) Use a função hstack do numpy para acrescentar uma coluna de 1s no X
    #     e formar a1
    # (3) Diferente dos slides, as amostra em X estão nas linhas e os atributos
    #     são as colunas. Portanto, para multiplicar pelos thetas use: X*theta.T
    

    
    
    
    
    
    
    
    

    ##########################################################################

    return z2, a2, z3, hx

print('\nEtapa forward propagation  ...\n')

z2, a2, z3, hx = forwardPropagation(X_train, initial_Theta1, initial_Theta2)

print('Saídas obtida para o primeiro dado em cada uma das classes: ')
for i in range( num_labels ):
    print('\tClasse %d -- hx: %1.7f' %(i+1, hx[0,i]))
    
print('\n\nAs saídas esperadas são, aproximadamente, as seguintes:') 
print('\tClasse 1 -- hx: 0.5190055') 
print('\tClasse 2 -- hx: 0.4760282') 
print('\tClasse 3 -- hx: 0.5697258') 
print('\tClasse 4 -- hx: 0.4565804') 
print('\tClasse 5 -- hx: 0.5227219') 
print('\tClasse 6 -- hx: 0.5821649') 
print('\tClasse 7 -- hx: 0.5725757') 
print('\tClasse 8 -- hx: 0.5066974') 
print('\tClasse 9 -- hx: 0.6033554') 
print('\tClasse 10 -- hx: 0.4879005') 

Iremos usar $h_\Theta(x^{(i)})$ computado acima para calcular o custo da rede neural. 

A função de custo (sem regularização) é descrita a seguir.

$$J(\Theta) = \frac{1}{m} \sum_{i=1}^{m}\sum_{k=1}^{K} \left[-y_k^{(i)} \log\left( \left(h_\Theta(x^{(i)})\right)_k \right) - \left(1 - y_k^{(i)}\right) \log \left( 1 - \left(h_\Theta(x^{(i)} )\right)_k \right)\right]$$

Na função $J$, a constante $K$ representa o número de classes. Assim, $h_\Theta(x^{(i)})_k$ corresponde à ativação (valor de saída) da $k$-ésima unidade de saída.

In [ ]:
def funcaoCusto(X, Y, Theta1, Theta2):
    '''
    Calcula o custo da rede neural com tres camadas
    voltada para tarefa de classificacao.

    Os parametros da rede neural sao colocados no vetor nn_params
    e precisam ser transformados de volta nas matrizes de peso.

    Parametros
    ----------   
    X : matriz com os dados de treinamento
    y : vetor com as classes dos dados de treinamento
    Theta1: pesos que ligam a camada de entrada à camada intermediária
    Theta2: pesos que ligam a camada intermediária à camada de saída
    
    Retorno
    -------
    J: valor do custo
    
    '''

    # estrai a quantidade de classes
    num_labels = len(np.unique(Y))
        
    # Qtde de amostras
    m = X.shape[0]
      
    # Usa a função forwardPropagation feita anteriormente  
    # Nesse exercício, você precisará usar apenas o hx retornado
    # pela função forwardPropagation 
    z2, a2, z3, hx = forwardPropagation(X, Theta1, Theta2)

    # inicializa o custo que você precisa calcular e retornar
    J = 0; 
    
    ########################## COMPLETE O CÓDIGO AQUI  ########################
    #
    # (1): use as saídas da rede (hx) obtidas com a função forwardPropagation e as classes
    #      reais binarizadas (Y) para calcular o custo 
    #


    
    
    
    
    
    
    


    ##########################################################################

    return J, z2, a2, z3, hx



print('\nFuncao de custo sem regularizacao ...\n')

J, z2, a2, z3, hx = funcaoCusto(X_train, Y_trainBin, initial_Theta1, initial_Theta2)

print('Custo obtido: %1.6f ' %J)

print('\nCusto esperado: 7.603099')

## Regularização

A função de custo com regularização é descrita a seguir.

$$J(\Theta) = \frac{1}{m} \sum_{i=1}^{m}\sum_{k=1}^{K} \left[-y_k^{(i)} \log\left( \left(h_\Theta(x^{(i)})\right)_k \right) - \left(1 - y_k^{(i)}\right) \log \left( 1 - \left(h_\Theta(x^{(i)} )\right)_k \right)\right]$$

$$ + \frac{\lambda}{2m} \left[\sum_{j=1}^{25} \sum_{k=1}^{400} \left(\Theta^{(1)}_{j,k}\right)^2 + \sum_{j=1}^{10} \sum_{k=1}^{25} \left(\Theta^{(2)}_{j,k}\right)^2\right]$$

É importante que a regularização não seja aplicada a termos relacionados aos *bias*. Neste contexto, estes termos estão na primeira coluna de cada matriz $\Theta^{(1)}$ e $\Theta^{(2)}$.

A seguir, complete a nova função de custo aplicando regularização.

In [ ]:
def funcaoCusto_reg(X, Y, Theta1, Theta2, vLambda):
    '''
    Calcula o custo da rede neural com tres camadas
    voltada para tarefa de classificacao.

    Os parametros da rede neural sao colocados no vetor nn_params
    e precisam ser transformados de volta nas matrizes de peso.

    Parametros
    ----------   
    X : matriz com os dados de treinamento
    y : vetor com as classes dos dados de treinamento
    Theta1: pesos que ligam a camada de entrada à camada intermediária
    Theta2: pesos que ligam a camada intermediária à camada de saída
    vLambda: parametro de regularizacao
    
    Retorno
    -------
    J: valor do custo
    
    '''

    # estrai a quantidade de classes
    num_labels = len(np.unique(Y))
        
    # Qtde de amostras
    m = X.shape[0]
        
    # Usa a função forwardPropagation feita anteriormente  
    # Nesse exercício, você precisará usar apenas o hx retornado
    # pela função forwardPropagation 
    z2, a2, z3, hx = forwardPropagation(X, Theta1, Theta2)

    # inicializa o custo que você precisa calcular e retornar
    J = 0; 
    
    ########################## COMPLETE O CÓDIGO AQUI  ########################
    # Instrucoes: Voce deve completar o codigo a partir daqui 
    #
    # (1): use as saídas da rede (hx) obtidas com a função forwardPropagation e as classes
    #      reais binarizadas (Ybin) para calcular o custo. 
    #      Obs: Você já fez isso no exercício anterior. Basta copiar de lá. 
    #
    # (2): Implemente a regularização na função de custo.
    

    
    
    
    
    
    
    
    
    
    
    
    ##########################################################################

    return J, z2, a2, z3, hx



# Parametro de regularizacao dos pesos
vLambda = 1;

J, z2, a2, z3, hx = funcaoCusto_reg(X_train, Y_trainBin, initial_Theta1, initial_Theta2, vLambda)

print('Custo obtidos: %1.6f ' %J)

print('Custo esperado: 7.615168')

## Backpropagation

Nesta parte do exercício, você implementará o algoritmo de *backpropagation* responsável por calcular o gradiente para a função de custo da rede neural. Terminada a implementação do cálculo do gradiente, você poderá treinar a rede neural minimizando a função de custo $J(\Theta)$ usando o algoritmo do gradiente descendente.

Primeiro, você precisará implementar o gradiente para a rede neural sem regularização. Após ter verificado que o cálculo do gradiente está correto, você implementará o gradiente para a rede neural com regularização.

Você deverá começar pela implementação do gradiente da sigmóide, o qual pode ser calculado utilizando a equação:

$$ g'(z) = \frac{d}{dz}g(z) = g(z)(1-g(z)), $$

sendo que

$$ g(z) = \frac{1}{1 + e^{-z}}. $$

Ao completar, teste diferentes valores para a função `sigmoidGradient`. Para valores grandes de *z* (tanto positivo, quanto negativo), o resultado deve ser próximo a zero. Quando $z = 0$, o resultado deve ser exatamente 0,25. A função deve funcionar com vetores e matrizes também. No caso de matrizes, a função deve calcular o gradiente para cada elemento.

In [ ]:
def sigmoidGradient(z):
    '''
    Retorna o gradiente da funcao sigmoidal para z 
    
    Calcula o gradiente da funcao sigmoidal
    para z. A funcao deve funcionar independente se z for matriz ou vetor.
    Nestes casos,  o gradiente deve ser calculado para cada elemento.
    '''
    
    g = np.zeros(z.shape)

    ########################## COMPLETE O CÓDIGO AQUI  ########################
    # Instrucoes: Calcula o gradiente da funcao sigmoidal para 
    #           cada valor de z (seja z matriz, escalar ou vetor).
    #


    
    
    
    
    
    
    ##########################################################################

    return g

print('\nAvaliando o gradiente da sigmoide...\n')

g = sigmoidGradient(np.array([1,-0.5, 0, 0.5, 1]))
print('Gradiente da sigmoide avaliado em [1 -0.5 0 0.5 1]:\n')
print(g)

print('\nSe sua implementacao estiver correta, o gradiente da sigmoide sera:')
print('[0.19661193 0.23500371 0.25 0.23500371 0.19661193]\n')

O objetivo do algoritmo *backpropagation* é encontrar a "parcela de responsabilidade" que cada neurônio da rede neural teve com o erro gerado na saída. Dada uma amostra de treino ($x^{(t)}, y^{(t)}$), primeiro é executado o passo de *feedforward* para calcular todas as ativações na rede, incluindo o valor de saída $h_{\Theta}(x)$. Posteriormente, para cada neurônio $j$ na camada $l$, é calculado o "erro" $\delta_{j}^{(l)}$ que mede o quanto determinado neurônio contribuiu para a diferença entre o valor esperado e o valor obtido na saída da rede.

Nos neurônios de saída, a diferença pode ser medida entre o valor esperado (rótulo da amostra) e o valor obtido (a ativação final da rede), onde tal diferença será usada para definir $\delta_{j}^{(3)}$ (visto que a camada 3 é a última). Nas camadas ocultas (quando houver mais de uma), o termo $\delta_{j}^{(l)}$ será calculado com base na média ponderada dos erros encontrados na camada posterior ($l + 1$).

A seguir, é descrito em detalhes como a implementação do algoritmo *backpropagation* deve ser feita. A descrição abaixo considera que iremos seguir os passos 1 a 4 dentro de um laço, processando uma amostra por vez. Mas, é possível também executar esses passos sem laço de repetição, usando apenas com operações matriciais. No passo 5, o gradiente acumulado é dividido pelas *m* amostras, o qual será utilizado na função de custo da rede neural.

 1. Coloque os valores na camada de entrada ($a^{(1)}$) para a amostra de treino a ser processada. Calcule as ativações das camadas 2 e 3 utilizando o passo de *feedforward*. Observe que será necessário adicionar um termo $+1$ para garantir que os vetores de ativação das camadas ($a^{(1)}$) e ($a^{(2)}$) também incluam o neurônio de *bias*.
 
 2. Para cada neurônio $k$ na camada 3 (camada de saída), defina:
    $$ \delta_{k}^{(3)} = (a^{(3)}_k - y_k), $$
    onde $y_k \in \{0,1\}$ indica se a amostra sendo processada pertence a classe $k$ ($y_k = 1$) ou não ($y_k = 0$).
    
 3. Para a camada oculta $l$ = 2, defina:

    $$ \delta^{(2)} = (\Theta^{(2)})^T \delta^{(3)}*g'(z^{(2)}) $$
    
 4. Acumule o gradiente usando a fórmula descrita a seguir. Lembre-se de não utilizar o valor associado ao bias $\delta^{(2)}_0$.
    
    $$ \Delta^{(l)} = \Delta^{(l)} + \delta^{(l+1)}(a^{(l)})^T $$
    
 5. Obtenha o gradiente sem regularização para a função de custo da rede neural dividindo os gradientes acumulados por $\frac{1}{m}$:
 
     $$ D^{(l)}_{ij} = \frac{1}{m}\Delta^{(l)}_{ij} $$

<br />
<br />

Neste ponto, iremos implementar o algoritmo de *backpropagation*. Vamos criar uma nova função de custo, baseada na função implementada anteriormente, que retorna as derivadas parciais dos parâmetros. Nesta função, iremos implementar o gradiente para a rede neural sem regularização.

In [ ]:
def funcaoCusto_backp(Theta1, Theta2, X, Y):
    '''
    Calcula o custo e gradiente da rede neural com tres camadas
    voltada para tarefa de classificacao.

    Os parametros da rede neural sao colocados no vetor nn_params
    e precisam ser transformados de volta nas matrizes de peso.
    
    Parametros
    ----------   
    X : matriz com os dados de treinamento
    y : vetor com as classes dos dados de treinamento
    Theta1: pesos que ligam a camada de entrada à camada intermediária
    Theta2: pesos que ligam a camada intermediária à camada de saída
    vLambda: parametro de regularizacao
    
    Retorno
    -------
    J: valor do custo
    
    Theta1_grad: derivadas parciais dos pesos da primeira camada.
    
    Theta1_grad: derivadas parciais dos pesos da segunda camada.
    '''

    # estrai a quantidade de classes
    num_labels = len(np.unique(Y))
    
    # Qtde de amostras
    m = X.shape[0]
    
    # retorna o custo J e as saídas de cada camada da rede (z2, a2, z3, hx)
    J, z2, a2, z3, hx = funcaoCusto(X, Y, Theta1, Theta2)  
    
    # As variaveis a seguir precisam ser retornadas corretamente
    Theta1_grad = np.zeros(Theta1.shape)
    Theta2_grad = np.zeros(Theta2.shape)  
       
    # executa a etapa backpropagation
    # ------------------------------------------
    
    delta3 = hx-Y
    
    delta2 = np.dot( delta3,Theta2[:,1:] )
    delta2 = delta2 * sigmoidGradient(z2)
    
    # a1 = x com uma coluna a mais com valor 1 para o bias
    a1 = np.hstack( [np.ones([m,1]),X] )
    
    # acumula o gradiente
    delta_cap2 = np.dot(delta3.T, a2) 
    delta_cap1 = np.dot(delta2.T, a1)

    # divide os gradientes acumulados por m
    Theta1_grad = delta_cap1 / m 
    Theta2_grad = delta_cap2 / m 
    # ------------------------------------------ 

    return J, Theta1_grad, Theta2_grad

# Parametro de regularizacao dos pesos.
vLambda = 3;

J, Theta1_grad, Theta2_grad = funcaoCusto_backp(initial_Theta1, initial_Theta2, X_train, Y_trainBin)

print(Theta1_grad.shape)
print(Theta2_grad.shape)

print('Segunda derivada parcial da primeira camada: %1.6f' %Theta1_grad[5,0])

print('Segunda derivada parcial da segunda camada: %1.6f' %Theta2_grad[5,0])

## Outra parte da regularização

Agora, a regularização deverá ser adicionada após se calcular o gradiente durante o algoritmo de *backpropagation*. Lembre-se que a regularização não é adicionada quando $j = 0$, ou seja, na primeira coluna de $\Theta$. Portanto, para $j \geq 1$, o gradiente é descrito como:

$$ D^{(l)}_{ij} = \frac{1}{m}\Delta^{(l)}_{ij} + \frac{\lambda}{m}\Theta^{(l)}_{ij} $$

<br/>
<br/>

Vamos criar uma nova função de custo que é uma atualização da função anterior, mas com regularização e gradiente.

In [ ]:
def funcaoCusto_backp_reg(Theta1, Theta2, X, Y, vLambda):
    '''
    Calcula o custo e gradiente da rede neural com tres camadas
    voltada para tarefa de classificacao.

    Os parametros da rede neural sao colocados no vetor nn_params
    e precisam ser transformados de volta nas matrizes de peso.
    
    Parametros
    ----------   
    X : matriz com os dados de treinamento
    y : vetor com as classes dos dados de treinamento
    Theta1: pesos que ligam a camada de entrada à camada intermediária
    Theta2: pesos que ligam a camada intermediária à camada de saída
    vLambda: parametro de regularizacao
    
    Retorno
    -------
    J: valor do custo
    
    grad: vetor que contem todas as derivadas parciais
          da rede neural.
    '''

    # Qtde de amostras
    m = X.shape[0]
         
    # As variaveis a seguir precisam ser retornadas corretamente
    J = 0;
    Theta1_grad = np.zeros(Theta1.shape)
    Theta2_grad = np.zeros(Theta2.shape)
    
    # retorna o custo J e as saídas de cada camada da rede (z2, a2, z3, hx)
    J, z2, a2, z3, hx = funcaoCusto_reg(X, Y, Theta1, Theta2, vLambda) 
    
    # backpropagation
    # ------------------------------------------
    
    delta3 = hx-Y
    
    delta2 = np.dot( delta3,Theta2[:,1:] )
    delta2 = delta2 * sigmoidGradient(z2) 
    
    # acumula o gradiente
    # --------------------------
    delta_cap2 = np.dot(delta3.T, a2) 
    
    # a1 = x com uma coluna a mais com valor 1 para o bias
    a1 = np.hstack( [np.ones([m,1]),X] )
    
    delta_cap1 = np.dot(delta2.T, a1)
    # --------------------------


    # divide os gradientes acumulados por m
    Theta1_grad = delta_cap1 / m 
    Theta2_grad = delta_cap2 / m 
    
    # acrescenta a regularização
    # A regularização não pode ser adicionada no bias. Ou seja, a coluna 0 do Theta não deve ser usada.
    Theta1_grad[:,1:] = Theta1_grad[:,1:] + vLambda / m * Theta1[:,1:]
    Theta2_grad[:,1:] = Theta2_grad[:,1:] + vLambda / m * Theta2[:,1:]
    # ------------------------------------------

    return J, Theta1_grad, Theta2_grad

# Parametro de regularizacao dos pesos.
vLambda = 6;

J, Theta1_grad, Theta2_grad = funcaoCusto_backp_reg(initial_Theta1, initial_Theta2, X_train, Y_trainBin, vLambda)

print('Segunda derivada parcial da primeira camada: %1.6f' %Theta1_grad[5,0])

print('Segunda derivada parcial da segunda camada: %1.6f' %Theta2_grad[5,0])

## Treinando a rede neural

Neste ponto, todo o código necessário para treinar a rede está pronto.
Agora, iremos iniciar o treinamento da rede neural usando o gradiente descendente para atualizar os pesos ao longo das épocas de treinamento.

In [ ]:
def gradienteDesc(X, Y, learnRate, MaxEpochs, vLambda, n_batches):

    # inicializa dos thetas
    Theta1 = inicializaPesosAleatorios(input_layer_size, hidden_layer_size, randomSeed = 10)
    Theta2 = inicializaPesosAleatorios(hidden_layer_size, num_labels, randomSeed = 20)  

    ########################## COMPLETE O CÓDIGO AQUI  ########################
    #
    # 1) Faça o algoritmo executar até atingir o limite de épocas
    # 2) Em cada época, execute um passo do gradiente usando mini-batch. Para separar
    #    em batches, use o método array_split da biblioteca numpy.     
    # 3) Use a função funcaoCusto_backp_reg para obter os gradientes\
    # 4) Atualize os thetas
    # 5) Atualize o histórico
    
    for i in range(MaxEpochs):

        if(i%100==0):
            print("Epoca: %d" % i)

        X_batches = np.array_split(X, n_batches)
        Y_batches = np.array_split(Y, n_batches)

        for j in range(n_batches):

            Xtemp = X_batches[j]
            Ytemp = Y_batches[j] 

            J, Theta1_grad, Theta2_grad = funcaoCusto_backp_reg(Theta1, Theta2, Xtemp, Ytemp, vLambda)

            Theta1 = Theta1 - (learnRate * Theta1_grad)
            Theta2 = Theta2 - (learnRate * Theta2_grad)

        ErrorHistory.append(J)   
    
    ##########################################################################
    
    return Theta1, Theta2, ErrorHistory
    

print('\nTreinando a rede neural.......')
print('.......(Aguarde, pois esse processo por ser um pouco demorado.)\n')

# Apos ter completado toda a tarefa, mude o parametro MaxEpochs para
# um valor maior e verifique como isso afeta o treinamento.
MaxEpochs = 2000

# Voce tambem pode testar valores diferentes para lambda.
vLambda = 1.5
    
ErrorHistory = []
learnRate = 0.1
n_batches=10

Theta1, Theta2, errorHistory = gradienteDesc(X_train, Y_trainBin, learnRate, MaxEpochs, vLambda, n_batches)

plt.plot(errorHistory)
plt.show()

## Visualizando os pesos


Uma das formas de entender o que a rede neural está aprendendo, é visualizar a representação capturada nos neurônios da camada oculta. Informalmente, dado um neurônio de uma camada oculta qualquer, uma das formas de visualizar o que esse neurônio calcula, é encontrar uma entrada *x* que o fará ser ativado (ou seja, um resultado próximo a 1). Para a rede neural que foi treinada, perceba que a $i$-ésima linha de $\Theta^{(1)}$ é um vetor com 785 dimensões, o qual representa os parâmetros para o $i$-ésimo neurônio. Se descartarmos o termo *bias*, teremos um vetor de 784 dimensões que representa o peso para cada pixel a partir da camada de entrada. Deste modo, uma das formas de visualizar a representação capturada pelo neurônio da camada oculta, é reorganizar essas 784 dimensões em uma imagem de 28 x 28 pixels e exibi-la. 

O script a seguir irá exibir uma imagem com 25 unidades, cada uma correspondendo a um neurônio da camada oculta.

In [ ]:
print('\nVisualizando a rede neural... \n')

print(X.shape)
print(Theta1[:, 1:].shape)

visualizaDados(Theta1[:, 1:])
plt.show()

## Predição

Após treinar a rede neural, ela será utilizada para predizer
os rótulos das amostras. Neste ponto, foi implementada a função de predição
para que a rede neural seja capaz de prever os rótulos no conjunto de dados
e calcular a acurácia do método.

In [ ]:
def predicao(X, Theta1, Theta2):
    '''
    Prediz o rotulo de X ao utilizar
    os pesos treinados na rede neural (Theta1, Theta2)
    '''
    
    m = X.shape[0] # número de amostras
    
    num_labels = 10 # quantidade de classes
        
    # calcula as saídas da rede (hx)
    z2, a2, z3, hx = forwardPropagation(X, Theta1, Theta2)
    
    # inicializa as predições
    pred = np.zeros( m, dtype=int )

    ########################## COMPLETE O CÓDIGO AQUI  ########################
    # Instrucoes: Voce deve completar o codigo a partir daqui 
    #
    # (1): use as saídas da rede (hx) obtidas com a função forwardPropagation e obtenha as classes. 
    #      Para cada dado, a classe deverá ser um dos seguintes valores: 0, 1, 2, 3, ..., 9
    #  
    # Obs: a classe predita pelo modelo é aquela que possui o maior valor de saída. Use o método
    #      argmax do numpy 


    
    
    
    
    
    ##########################################################################
    
    return pred
   
pred = predicao(X_test, Theta1, Theta2)

print('\nAcuracia no conjunto de teste: %1.2f%%\n'%( np.mean( pred == Y_test ) * 100) )

print('\nAcuracia esperada: 89.80% (aproximadamente)\n')